### Title

### Introduction

A group of UBC students have created their own special minecraft server. The purpose of the server is to gather data about the players, and how they play the game to gain possible insights on machine learning and AI. The students have provided us with data about the individual players and logged sessions on the server. This data is in the form of two .csv (comma separated values) files - players.csv and sessions.csv. In this project we will highlight a specific question we have formulated about relationships, patterns or correlations between the data to find out specific information on the players and/or sessions. The conclusions of our projects will be provided to the owners of the server and used to assist in their study about AI and machine learning. Our topic of interest within the data sets is about predicting which “kinds” of players are most likely to contribute a large amount of data. What this topic really means is which types of players had the most total played hours on the minecraft server. Based on this topic, we have formulated a specific question to determine if there is any sort of correlation between the observations of variables within the dataset and the observations of the total played hours variable. We proposed, "Can gender, experience level, and age predict played_hours in the players.csv dataset?", as our predictive question. Throughout the report, we will use a series of data analysis, including visualization and modeling, to help answer the predictive question.


The players.csv dataset contains seven total variables, four of them being “played_hours”, “Age”, “gender” and “experience”. The following table has been created to summarize the 7 variables in the players.csv dataset. We must check if there are any N/A values in the observations of the players.csv dataset. This is necessary because when performing models and data visualizations in R, if there are any N/A values within the variables of interest it could possibly results in an error in the code. The following code reveals that within the "Age" column there are 2 N/A's. Therefore in future code we must make sure to remove these N/A's so we don't receive an error. 

There are 196 observations in the player.csv dataset


In [ ]:
# loads data 
# wrangles and cleans the data to the format necessary for the planned analysis
# performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
# creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
# performs the data analysis
# creates a visualization of the analysis 
# note: all figures should have a figure number and a legend




In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
library(janitor)

In [ ]:
# Load and Wrangle
data <- read_csv("data/players.csv") |> 
clean_names() |>
mutate(experience = factor(experience, levels = c("Beginner", "Amateur", "Regular", "Veteran", "Pro"), ordered = TRUE)) |>
mutate(gender = fct_collapse(gender, Other = c("Agender", "Two-Spirited", "Other", "Prefer not to say", "Non-binary")))|>
mutate(gender = factor(gender, levels = c("Male", "Female", "Other"))) # lump two_spirit and agenedr with other due to limited occurnaces

# Summary
sum_data <- data |>
summarize(n = n(), 
          mean_played = mean(played_hours), 
          sd_played = sd(played_hours), 
          min_age = min(age, na.rm = TRUE), 
          max_age = max(age, na.rm = TRUE))

sum_data_by_exp <- data |>
group_by(experience) |>
summarize(n = n(), 
          mean_played = mean(played_hours), 
          mean_age = mean(age, na.rm = TRUE))

sum_data_by_gender <- data |>
group_by(gender) |>
summarize(n = n(), 
          mean_played = mean(played_hours), 
          mean_age = mean(age, na.rm = TRUE))

sum_data
sum_data_by_exp
sum_data_by_gender



In [ ]:
p1_full <- ggplot(data, aes(played_hours)) +
geom_histogram(bins = 40, color = "black", fill = "skyblue") +
labs(
    title = "Figure 1A: Full Distribution of Played Hours",
    x = "Played Hours",
    y = "Count"
) +
theme_minimal()

p95 <- quantile(data$played_hours, 0.95, na.rm = TRUE)
p1_zoom <- ggplot(data |> filter(played_hours <= p95), aes(played_hours)) +
geom_histogram(bins = 40, color = "black", fill = "steelblue") +
labs(
    title = "Figure 1B: Zoomed Distribution (0–95th percentile)",
    x = "Played Hours",
    y = "Count"
) +
theme_minimal()

p1_full
p1_zoom


# Used a log scale for large discrepancies within y values so that we can actually see the boxplots
p2 <- ggplot(data, aes(experience, played_hours, fill = experience)) +
geom_boxplot(outlier.alpha = 0.4) +
scale_y_log10() +
labs(
    title = "Figure 2: Played Hours by Experience (Log Scale)",
    x = "Experience",
    y = "Played Hours (log scale)",
    fill = "Experience"
) +
theme_minimal()

p2

p3 <- ggplot(data, aes(gender, played_hours, fill = gender)) +
geom_boxplot(outlier.alpha = 0.4) +
scale_y_log10() +
labs(
    title = "Figure 3: Played Hours by Gender (Log Scale)",
    x = "Gender",
    y = "Played Hours (log scale)",
    fill = "Gender"
) +
theme_minimal()

p3

p4 <- ggplot(data, aes(x = age, y = played_hours, color = experience)) +
geom_point(alpha = 0.7) +
labs(
    title = "Figure 4: Age vs Played Hours",
    x = "Age",
    y = "Played Hours",
    color = "Experience"
) +
theme_minimal()
p4

In [ ]:
# Knn  !!! only age has NA values
model_data <- data |>
  select(played_hours, age, experience, gender)

split <- initial_split(model_data, prop = 0.8) # no strata since catagorial, used 0.8 split since it is standard for large datasets
training_data <- training(split)
testing_data  <- testing(split)


knn_recipe <- recipe(played_hours ~ ., data = training_data) |>
  step_impute_median(all_numeric_predictors()) |> # imputes/compensates for NA values in age
  step_novel(all_nominal_predictors()) |>   # this
  step_dummy(all_nominal_predictors(), one_hot = TRUE) |> # and this ensures the data always accounts for all gender/experience values
  step_zv(all_predictors()) |>
  step_normalize(all_predictors())

knn_model <- nearest_neighbor(mode = "regression", neighbors = tune(), weight_func = "rectangular") |>
  set_engine("kknn") |>
  set_mode("regression") 

data_vfold <- vfold_cv(training_data, v = 5)  # used 5 folds since we have small gorups of categorial predictors
k_grid <- tibble(neighbors = seq(3, 25, by = 2)) # 2 step to avoid excessive computation, range 3to 25 kinda arbitrary but is a good general rnage



knn_workflow <- workflow() |>
  add_model(knn_model) |>
  add_recipe(knn_recipe)

tune_results <- tune_grid(knn_workflow, resamples = data_vfold, grid = k_grid, metrics = metric_set(rmse, rsq, mae))

best_k <- select_best(tune_results, "rmse")
best_k

final_knn <- finalize_workflow(knn_workflow, best_k)

final_knn_fit <- fit(final_knn, data = training_data)

preds <- predict(final_knn_fit, testing_data) |>
  bind_cols(testing_data)

metrics(preds, truth = played_hours, estimate = .pred)

In [ ]:
pred_plot <- ggplot(preds, aes(x = played_hours, y = .pred)) +
  geom_point(alpha = 0.6, color = "blue") +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "red") +
  labs(
    title = "Figure 5: KNN Predicted vs Actual Played Hours",
    x = "Actual Played Hours",
    y = "Predicted Played Hours"
  ) +
  theme_minimal()

preds_clean <- preds |>
  filter(played_hours < 100)



pred_plot_clean <- ggplot(preds_clean, aes(x = played_hours, y = .pred)) +
  geom_point(alpha = 0.6, color = "blue") +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "red") +
  labs(
    title = "Figure 5: KNN Predicted vs Actual Played Hours",
    x = "Actual Played Hours",
    y = "Predicted Played Hours"
  ) +
  theme_minimal()
pred_plot_clean
pred_plot

In [ ]:
linreg_model <- linear_reg() |>
  set_engine("lm") |>
  set_mode("regression")

linreg_workflow <- workflow() |>
  add_model(linreg_model) |>
  add_recipe(
    recipe(played_hours ~ ., data = training_data) |>
      step_novel(all_nominal_predictors()) |>
      step_dummy(all_nominal_predictors(), one_hot = TRUE) |>
      step_zv(all_predictors()) |>
      step_normalize(all_predictors()) |>
      step_impute_median(all_numeric_predictors())  # handle age NAs
  )

linreg_fit <- fit(linreg_workflow, data = training_data)

preds_linreg <- predict(linreg_fit, testing_data) |>
  bind_cols(testing_data)

metrics(preds_linreg, truth = played_hours, estimate = .pred)


In [ ]:
preds_clean_lin <- preds_linreg |>
  filter(played_hours < 100)

pred_plot_lin <- ggplot(preds_clean_lin, aes(x = played_hours, y = .pred)) +
  geom_point(alpha = 0.6, color = "blue") +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "red") +
  labs(
    title = "Figure 5: Linear Reg Predicted vs Actual Played Hours",
    x = "Actual Played Hours",
    y = "Predicted Played Hours"
  ) +
  theme_minimal()

pred_plot_lin